In [1]:
import tensorflow as tf
import numpy as np
from tdc.single_pred import Tox
from rdkit import Chem

2023-12-15 14:22:46.532379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 14:22:46.654480: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 14:22:47.461632: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.8/lib64:/usr/local/cuda-11.3/lib64
2023-12-15 14:22:47.461729: W te

In [13]:
from rdkit.Chem import MolFromSmiles, MolToSmiles
from tqdm import tqdm
from SmilesPE import tokenizer
def encode(smiles_list,labels):
    mols = []
    res_labels = []
    len_list = []
    
    for smiles in smiles_list:
        mol = MolFromSmiles(smiles)
        mols.append(mol)
    res = []
    for index1,mol in (enumerate(tqdm(mols))):
        index = 0
        res_temp = []
        while(len(res_temp)<20 and index <100):
            random_smiles = MolToSmiles(mol, doRandom=True) # e.g 'C(C(=O)O)N' 
            res_temp.append(random_smiles)
            res_temp = list(set(res_temp))
            index+=1
        res += res_temp
        res_labels += [labels[index1]]*len(res_temp)
        len_list.append(len(res_temp))
        
    # mapping atomID into SMILES srting
    res_train = []


    for smiles in tqdm(res):
        smile_tokens = tokenizer.atomwise_tokenizer(smiles)
        res_train.append(smile_tokens)

    return res_train,res_labels,len_list

In [14]:
Tox_list = ['hERG_Karim','LD50_Zhu','hERG','AMES','DILI','Skin Reaction','Carcinogens_Lagunin','ClinTox']

In [15]:
tox_train,tox_label = Tox(name='herg').get_data(format='DeepPurpose')

Found local copy...
Loading...
Done!


In [16]:
def create_variable(var_name):
    globals()[var_name] = []
def get_variable_value(var_name):
    return globals().get(var_name)


In [18]:
import pickle
for tox in Tox_list:
    create_variable(tox)
    tox_train,tox_label = Tox(name=tox).get_data(format='DeepPurpose')
    temp_train = get_variable_value(tox)
    res = encode(tox_train,tox_label)
    temp_train.append(res)
    with open('./SMILE_Tox_data/'+tox,'wb') as file:
        pickle.dump(temp_train,file)


Found local copy...
Loading...
Done!
100%|██████████| 268795/268795 [00:02<00:00, 92673.42it/s] 
Found local copy...
Loading...
Done!
100%|██████████| 141226/141226 [00:01<00:00, 138356.58it/s]
Found local copy...
Loading...
Done!
[14:27:56] WARNING: not removing hydrogen atom without neighbors
[14:27:56] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 13092/13092 [00:00<00:00, 114367.48it/s]
Found local copy...
Loading...
Done!
100%|██████████| 139900/139900 [00:01<00:00, 102172.08it/s]
Found local copy...
Loading...
Done!
100%|██████████| 9404/9404 [00:00<00:00, 127525.02it/s]
Found local copy...
Loading...
Done!
100%|██████████| 7712/7712 [00:00<00:00, 178246.94it/s]
Found local copy...
Loading...
Done!
100%|██████████| 5501/5501 [00:00<00:00, 111902.62it/s]
Found local copy...
Loading...
Done!
100%|██████████| 29009/29009 [00:00<00:00, 109433.62it/s]


In [19]:
from tdc.utils import retrieve_label_name_list
import pickle
Tox_list_label = ['herg_central','Tox21','ToxCast']
for tox in Tox_list_label:
    label_list = retrieve_label_name_list(tox)
    for label in label_list:

        create_variable(tox+'_'+label)
        tox_train,tox_label = Tox(name=tox,label_name=label).get_data(format='DeepPurpose')
        temp_train = get_variable_value(tox+'_'+label)
        temp_train.append(encode(tox_train,tox_label))
        with open('./SMILE_Tox_data/'+tox+'_'+label,'wb') as file:
            pickle.dump(temp_train,file)

Found local copy...
Loading...
Done!
100%|██████████| 6137270/6137270 [00:51<00:00, 118839.73it/s]
Found local copy...
Loading...
Done!
